In [1]:
#Workshop 14: เกม Save the Witch

import sys, pygame, random
from pygame.locals import *

SCREEN_W = 600
SCREEN_H = 400
BLUESKY = (200, 220, 255)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (150, 150, 150)
FPS = 30

pygame.init()
pygame.display.set_caption('SAVE the WITCH By Pantawat')
screen = pygame.display.set_mode((SCREEN_W, SCREEN_H))
screen_rect = screen.get_rect()

clock = pygame.time.Clock()
img_path = r'assets/pygame/images'
font_path = r'assets/pygame/fonts'
snd_path = r'assets/pygame/sounds'

win = False
blood_end = False
game_over = False

total_weapons = 50      #จำนวนอาวุธทั้งหมดที่มี (เลือกกำหนดค่าตามต้องการ)
shootout_weapons = 0    #จำนวนอาวุธที่ถูกปล่อยออกไปแล้ว
killed_weapons = 0      #จำนวนอาวุธที่หลุดขอบหน้าจอออกไปแล้ว
                        #ซึ่งจะใช้ติดสินผลแพ้ชนะอีกอย่างหนึ่ง


#--------------- Witch Sprite ---------------
the_witch_img = pygame.image.load(fr'{img_path}/witch.png')
the_witch_img = pygame.transform.scale(the_witch_img, (64, 64))

class Witch(pygame.sprite.Sprite):
    def __init__(self):
        super(Witch, self).__init__()
        self.image = the_witch_img
        self.rect = self.image.get_rect(center=screen_rect.center) 
        self.distance = 5

    def update(self, keys):
        if keys[K_RIGHT]: 
            self.rect.move_ip(self.distance, 0)
            if self.rect.right >  SCREEN_W:
                self.rect.right = SCREEN_W

        elif keys[K_LEFT]: 
            self.rect.move_ip(-self.distance, 0)
            if self.rect.left < 0:
                self.rect.left = 0           

        elif keys[K_UP]: 
            self.rect.move_ip(0, -self.distance)
            if self.rect.top < 0:
                self.rect.top = 0      

        elif keys[K_DOWN]: 
            self.rect.move_ip(0, self.distance)
            if self.rect.bottom > SCREEN_H:
                self.rect.bottom = SCREEN_H  


#--------------- Weapon Sprite ---------------
#อาวุธมี 3 อย่างคือ ลูกศร หอก และดาบ โดยแต่ละอย่างมีความเร็วต่างกัน
arrow = pygame.image.load(fr'{img_path}/arrow.png')  #ลูกศร
spear = pygame.image.load(fr'{img_path}/spear.png')  #หอก
sword = pygame.image.load(fr'{img_path}/sword.png')  #ดาบ
weapons = [[arrow, 15], [spear, 10], [sword, 7]]     #ภาพ, ความเร็ว

class Weapon(pygame.sprite.Sprite):
    def __init__(self):
        super(Weapon, self).__init__()
        r = random.choice(weapons)  
        self.image = r[0]   
        self.speed = r[1]
        self.image = pygame.transform.scale(self.image, (32, 32))

        h = self.image.get_height()
        start_left = SCREEN_W + random.randint(0, 500)
        start_top =  random.randint(h//2, (SCREEN_H - h))
        self.rect = self.image.get_rect(topleft=(start_left, start_top))

    def update(self):
        global killed_weapons, game_over, win
        self.rect.move_ip(-self.speed, 0)
        if self.rect.right < 0:
            self.kill()
            killed_weapons += 1

            #ถ้าอาวุธหลุดกรอบหน้าไปแล้วทั้งหมด ให้จบเกมแบบผู้ชนะ
            if killed_weapons == total_weapons:
                game_over = True
                win = True


#--------------- Cloud Sprite ---------------
cloud1 = pygame.image.load(fr'{img_path}/cloud1.png')
cloud2 = pygame.image.load(fr'{img_path}/cloud2.png')
cloud3 = pygame.image.load(fr'{img_path}/cloud3.png')
clouds = [cloud1, cloud2, cloud3]

class Cloud(pygame.sprite.Sprite):
    def __init__(self):
        super(Cloud, self).__init__()
        self.image = random.choice(clouds)
        h = self.image.get_height()
        start_left = SCREEN_W + random.randint(0, 500)
        start_top =  random.randint(h//2, (SCREEN_H - h))
        self.rect = self.image.get_rect(topleft=(start_left, start_top))
        self.speed = 5

    def update(self):
        self.rect.move_ip(-self.speed, 0)
        if self.rect.right < 0:
            self.kill()


#--------------- Blood Sprite Sheet ---------------
#ภาพเลือดกระเด็น แบ่งเป็น 6 ภาพย่อยแบบแถวเดียว
bld_img =  pygame.image.load(fr'{img_path}/blood.png') 
bld_num_subimgs = 6            
bld_subimg_w = bld_img.get_width() // bld_num_subimgs 
bld_subimg_h = bld_img.get_height() 
bld_subimgs = []        

for i in range(bld_num_subimgs):
    x = i * bld_subimg_w
    img = bld_img.subsurface(x, 0, bld_subimg_w, bld_subimg_h)
    bld_subimgs.append(img)

bld_num_subimgs = len(bld_subimgs)          
bld_repeat = FPS // bld_num_subimgs  
bld_last_frame = (bld_repeat * bld_num_subimgs) - 1    

class Blood(pygame.sprite.Sprite):
    def __init__(self, pos):       
        super(Blood, self).__init__()
        self.image = bld_subimgs[0]
        left = pos[0] - (bld_subimg_w // 2)
        top = pos[1] - (bld_subimg_h // 2)
        self.rect = pygame.Rect(
            left, top, bld_subimg_w, bld_subimg_w
        )
        self.index = 0

    def update(self):
        global blood_end, game_over, playing
        if self.index >= bld_last_frame:
            self.kill()
            #เมื่อแสดงภาพเลือดครบทุกภาพย่อย
            #ให้ถือว่าจบเกม
            blood_end = True
            game_over = True
        else:
            i = self.index // bld_repeat
            self.image = bld_subimgs[i]
            screen.blit(self.image, self.rect)
            self.index += 1


#--------------- Images & Sounds ---------------
#ภาพหน้าปก ซึ่งจะแสดงเป็นพื้นหลังของหน้าจอเริ่มต้น
cover = pygame.image.load(fr'{img_path}/save-the-witch-cover.png')

intro_snd = pygame.mixer.Sound(fr'{snd_path}/one-thousand.wav')
the_witch_bg_snd = pygame.mixer.Sound(fr'{snd_path}/the-witch.wav')
the_witch_laugh = pygame.mixer.Sound(fr'{snd_path}/the-witch-laugh.wav')                 
the_witch_scream = pygame.mixer.Sound(fr'{snd_path}/the-witch-scream.wav') 


#--------------- Text & Intro Screen ---------------
def draw_text(text, size, colour, x, y, fontfile=None):
    #ถ้าไม่ระบุชื่อไฟล์ของฟอนต์ ให้ใช้ฟอนต์ของระบบ
    if fontfile == None:
        font = pygame.font.SysFont(None, size)
    else:
        font = pygame.font.Font(fontfile, size)

    text_surface = font.render(text, True, colour)
    text_rect = text_surface.get_rect(midtop=(x, y))
    screen.blit(text_surface, text_rect)

def intro_screen():
    intro_snd.play(-1)
    screen.fill(BLUE)
    screen.blit(cover, cover.get_rect())    #วาดภาพหน้าปก

    #ชื่อเกม ใช้ฟอนต์ที่ชื่อ  the-witch
    draw_text("SAVE the WITCH", 36, WHITE, 200, 30, 
             fr'{font_path}/the-witch.ttf'
    )
    draw_text("Use arrow keys to move the witch", 24, RED, 200, 200)

    #แสดงปุ่ม START
    btn_start_image = pygame.image.load(fr'{img_path}/btn-start-2.png')
    btn_start_rect = btn_start_image.get_rect(left=80, top=260)

    #แสดงปุ่ม EXIT
    btn_exit_image = pygame.image.load(fr'{img_path}/btn-exit-2.png')
    btn_exit_rect = btn_exit_image.get_rect(left=80, top=320)

    screen.blit(btn_start_image, btn_start_rect)
    screen.blit(btn_exit_image, btn_exit_rect)

    pygame.display.flip()

    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == MOUSEBUTTONDOWN:
                #ถ้าคลิกที่ปุ่ม START ให้เริ่มเล่นเกม
                if btn_start_rect.collidepoint(pygame.mouse.get_pos()):
                    intro_snd.stop()
                    waiting = False
                #ถ้าคลิกปุ่ม EXIT ให้ออกจากเกม
                elif btn_exit_rect.collidepoint(pygame.mouse.get_pos()):
                    pygame.quit()
                    sys.exit()    


#---------------------------------------------
playing = False
running = True
while running:
    if not playing:
        intro_screen()

        playing = True

        #เราต้องกำหนดค่าตัวแปรพร้อมอินสแตนซ์ที่นี่
        #เพื่อให้ค่าเริ่มต้นถูกรีเซตทุกครั้งที่จะเริ่มเกมใหม่
        win = False
        blood_end = False
        game_over = False
        total_weapons = 50      
        shootout_weapons = 0    
        killed_weapons = 0      
                                
        group_weapon = pygame.sprite.Group()
        group_cloud = pygame.sprite.Group()

        group_witch = pygame.sprite.Group()
        witch = Witch()
        group_witch.add(witch)

        ADD_WEAPON = pygame.USEREVENT + 1
        pygame.time.set_timer(ADD_WEAPON, 1000)

        ADD_CLOUD = pygame.USEREVENT + 2
        pygame.time.set_timer(ADD_CLOUD, 2500)

        group_blood = pygame.sprite.Group()

        the_witch_bg_snd.play(-1)

        the_witch_laughing = False  
        #ตัวแปรนี้ใช้เทียบว่ากำลังเล่นไฟล์เสียงหัวเราะอยู๋หรือไม่

    for event in pygame.event.get():
        if event.type == QUIT:
            running = False    
            pygame.quit()
            sys.exit()
        elif event.type == ADD_WEAPON and not game_over:
            #ถ้าอาวุธยังถูกปล่อยมาไม่หมด ก็สร้างเพิ่มไปเรื่อยๆ
            if shootout_weapons < total_weapons:
                new_weapon = Weapon()
                group_weapon.add(new_weapon)
                shootout_weapons += 1   #นับจำนวนอาวุธที่ถูกปล่อยออกมา
        elif event.type == ADD_CLOUD and not game_over:
            new_cloud = Cloud()
            group_cloud.add(new_cloud)
        
        #ถ้าคลิกเมาส์เมื่อจบเกมแล้ว ให้เข้าสู่หน้าจอเริ่มต้น
        elif event.type == MOUSEBUTTONDOWN and game_over:
            playing = False

    screen.fill(BLUESKY)

    keys = pygame.key.get_pressed()
    witch.update(keys)
    group_weapon.update()
    group_cloud.update()
    group_blood.update()
    
    group_witch.draw(screen)
    group_weapon.draw(screen)
    group_cloud.draw(screen)
    group_blood.draw(screen)

    #ถ้าจบเกม ให้แสดงข้อความว่าเป็นผู้แพ้หรือชนะ
    if game_over: 
        the_witch_bg_snd.stop()  #หยุดเล่นเสียงเบื้องหลัง  
        if blood_end: #ถ้าแสดงภาพเลือดครบแล้ว ให้แสดงข้อความว่าแพ้
            draw_text('You Lose', 60, RED, screen_rect.centerx, 150)
        elif win:     #ถ้าชนะ ให้แสดงข้อความพร้อมเล่นไฟล์เสียงหัวเราะ
            draw_text('You Win', 60, BLUE, screen_rect.centerx, 150)
            #เราต้องตรวจสอบก่อนว่าขณะนั้นกำลังเล่นไฟล์อยู่แล้วหรือไม่
            #มิฉะนั้นไฟล์จะถูกเล่นซ้ำๆ ในทุกเฟรม
            if not the_witch_laughing:
                the_witch_laugh.play()
                the_witch_laughing = True
            
        if blood_end or win:
            draw_text('Click mouse to continue...', 24, BLACK, 
                        screen_rect.centerx, 300)
    
    #ถ้ายังไม่จบเกม ให้แสดงจำนวนอาวุธที่ถูกปล่อยออกไปแล้ว / จำนวนทั้งหมด
    else:
        text = f'weapon: {shootout_weapons} / {total_weapons}'
        draw_text(text, 24, GRAY, screen_rect.centerx, (SCREEN_H - 30))


    pygame.display.flip()
    clock.tick(FPS)

    #ตรวจสอบการชนแบบกลุ่ม
    hits = pygame.sprite.groupcollide(
            group_witch, group_weapon, True, True, pygame.sprite.collide_mask
    )
    if len(hits) > 0:
        for hit in hits:
            first_hit = hit
            break

        center = hit.rect.center
        blood = Blood(center)
        group_blood.add(blood)
        the_witch_scream.play()    
        game_over = True
        

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


2023-09-28 17:42:22.701 Python[3175:123332] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


SystemExit: 

/Users/pantawatchamrongpan/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 

In [ ]:
%pip install pygame